# Data Cleaning for Route Solutions

Date: Jan 17, 2022

This notebook was used for developing the data cleaning script for data exported from Routific.

Routific is an app that is used to optimize delivery routes for the volunteer drivers.

Note: The data shown below is dummy data generated for demonstration purposes only.

### Import Packages and Data

In [36]:
import pandas as pd
import numpy as np
import re


In [37]:
# Import Data
path = "../data/raw/"
file_name = "Routific Solution Dummy Data - Other 2023-01-17.csv"

raw_data = pd.read_csv(path + file_name)

raw_data.head()


,Driver Name,Driver Phone,Stop Number,Visit Name,Address,Street,City,State,Zip code,Latitude,...,Completion/skipped Time,Skipped Reason,Driver Notes,FamilyID,Alternate Phone,Toy Box Summary,suite,Chicken,Vegetarian,Allergies
0,John Doe,1234567890,0,NaN,"1 Main St, M1A 2B3",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,John Doe,1234567890,1,Adam Smith,"2 Main St, M1A 2B3",NaN,NaN,NaN,NaN,NaN,...,10:20,Not home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Doe,1234567890,2,Beatrice Smith,"2 Main St, M1A 2B4",NaN,NaN,NaN,NaN,NaN,...,10:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,John Doe,1234567890,3,Candice Smith,"2 Main St, M1A 2B5",NaN,NaN,NaN,NaN,NaN,...,10:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,John Doe,1234567890,4,David Smith,"2 Main St, M1A 2B6",NaN,NaN,NaN,NaN,NaN,...,10:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# View raw data columns
raw_data.columns


Index(['Driver Name', 'Driver Phone', 'Stop Number', 'Visit Name', 'Address',
       'Street', 'City', 'State', 'Zip code', 'Latitude', 'Longitude',
       'Time window start', 'Time window end', 'Arrive at', 'Start at',
       'Finish by', 'Duration', 'Idle time', 'Distance(km)', 'Load', 'Phone',
       'Email', 'Types', 'Notes', 'Notes 2', 'Photo Url', 'Stop Status',
       'Completion/skipped Time', 'Skipped Reason', 'Driver Notes', 'FamilyID',
       'Alternate Phone', 'Toy Box Summary', 'suite', 'Chicken', 'Vegetarian',
       'Allergies'],
      dtype='object')

### Select Columns

Some of the columns are empty or are not relevant for our route analysis, so we'll keep only the columns we need.

In [39]:
cols = [
    "Driver Name",
    "Stop Number",
    "Visit Name",
    "Address",
    "Start at",
    "Finish by",
    "Distance(km)",
    "Stop Status",
    "Completion/skipped Time",
    "Skipped Reason",
    "Driver Notes",
    "FamilyID",
]

# Clean column names for upload (letters, numbers, underscores only)
col_names = [re.sub("(\s|/|\(|\))", "_", s) for s in cols]

# Select required columns and rename
raw_data = raw_data[cols]
raw_data.columns = col_names


raw_data


,Driver_Name,Stop_Number,Visit_Name,Address,Start_at,Finish_by,Distance_km_,Stop_Status,Completion_skipped_Time,Skipped_Reason,Driver_Notes,FamilyID
0,John Doe,0,NaN,"1 Main St, M1A 2B3",10:00,NaN,0,NaN,NaN,NaN,NaN,NaN
1,John Doe,1,Adam Smith,"2 Main St, M1A 2B3",10:10,10:20,2,skipped,10:20,Not home,NaN,NaN
2,John Doe,2,Beatrice Smith,"2 Main St, M1A 2B4",10:20,10:30,2,done,10:30,NaN,NaN,NaN
3,John Doe,3,Candice Smith,"2 Main St, M1A 2B5",10:30,10:40,2,done,10:40,NaN,NaN,NaN
4,John Doe,4,David Smith,"2 Main St, M1A 2B6",10:40,10:50,2,done,10:50,NaN,NaN,NaN
5,John Doe,5,Ethan Smith,"2 Main St, M1A 2B7",10:50,11:00,2,skipped,11:00,Not home,NaN,NaN
6,John Doe,999,NaN,"3 Main St, M1A 2B3",11:30,NaN,10,NaN,NaN,NaN,NaN,NaN
7,Mary Doe,0,NaN,"1 Maple St, M1A 2B3",10:00,NaN,0,NaN,NaN,NaN,NaN,NaN
8,Mary Doe,1,Adam Smith,"1 Maple St, M1A 2B4",10:10,10:20,2,NaN,NaN,NaN,NaN,NaN
9,Mary Doe,2,Beatrice Smith,"1 Maple St, M1A 2B5",10:20,10:30,2,NaN,NaN,NaN,NaN,NaN


### Create new features

From `Address`:
- Postal Code
- FSA (First 3 Digits of Postal Code)

From `Stop Status`:
- Stop Completion (binary)
  
From `File Name`:
- Dispatch Date
- Dispatch Type
- Dispatch Name (Date + Type)

From `File Name`:
- Dispatch Date
- Dispatch Type
- Dispatch Name (Date + Type)

In [40]:
clean_data = raw_data.copy()

# Create new address variables
clean_data["FSA"] = clean_data["Address"].str.extract(
    r"([ABCEGHJ-NPRSTVXY]\d[ABCEGHJ-NPRSTV-Z])"
)

clean_data["Postal_Code"] = clean_data["Address"].str.extract(
    r"([ABCEGHJ-NPRSTVXY]\d[ABCEGHJ-NPRSTV-Z][ -]?\d[ABCEGHJ-NPRSTV-Z]\d)"
)

# Create binarized Stop Completion variable
clean_data["Stop_Completion"] = np.where(
    clean_data["Stop_Status"] == "done",
    "1",
    np.where(
        clean_data["Stop_Status"] == "skipped",
        "0",
        np.where(
            (
                clean_data["Stop_Status"].isna()
                & clean_data["Driver_Notes"].isna()
                & clean_data["Distance_km_"].notna()
            ),
            "1",  # If driver forgets to mark status and didn't leave note, assume done
            None,
        ),
    ),
).astype("int")

# Create File Name variable to extract dispatch information
clean_data["File_Name"] = file_name

clean_data["Dispatch_Date"] = clean_data["File_Name"].str.extract(
    r"(\d{4}-\d{2}-\d{2})"
)

clean_data["Dispatch_Type"] = np.where(
    clean_data["File_Name"].str.contains("FSP"),
    "FSP",
    np.where(
        clean_data["File_Name"].str.contains("MOP"),
        "MOP",
        np.where(clean_data["File_Name"].str.contains("Special"), "Special", "Other"),
    ),
)

clean_data["Dispatch_Name"] = clean_data["Dispatch_Date"].str.cat(
    clean_data["Dispatch_Type"], sep=" "
)


### Clean Start & Stop Times

Some data points in the `Start_at` and `Finish_by` columns have times that exceed 24 hours. These points are most likely due to the driver forgetting to record the last stop in the app, then closing it within the next few days.
We'll exclude time points with hours greater than 24 (e.g. "92:10"). Then, we'll convert these times to datetimes in Pandas to calculate the duration.

In [41]:
# Remove Start/Finish times exceeding 24 hours
clean_data["Start_at"] = np.where(
    clean_data["Start_at"].str[0:2].astype(float) > 23, None, clean_data["Start_at"]
)
clean_data["Finish_by"] = np.where(
    clean_data["Finish_by"].str[0:2].astype(float) > 23, None, clean_data["Finish_by"]
)

# Convert times to Datetime formats and calculate duration between stops in minutes
clean_data["Start_at"] = pd.to_datetime(clean_data["Dispatch_Date"]).dt.tz_localize(
    "US/Eastern"
) + pd.to_timedelta(clean_data["Start_at"] + ":00")
clean_data["Finish_by"] = pd.to_datetime(clean_data["Dispatch_Date"]).dt.tz_localize(
    "US/Eastern"
) + pd.to_timedelta(clean_data["Finish_by"] + ":00")
clean_data["Duration_mins"] = clean_data.groupby("Driver_Name")[
    "Start_at"
].diff() / pd.Timedelta(minutes=1)


In [42]:
clean_data


,Driver_Name,Stop_Number,Visit_Name,Address,Start_at,Finish_by,Distance_km_,Stop_Status,Completion_skipped_Time,Skipped_Reason,Driver_Notes,FamilyID,FSA,Postal_Code,Stop_Completion,File_Name,Dispatch_Date,Dispatch_Type,Dispatch_Name,Duration_mins
0,John Doe,0,NaN,"1 Main St, M1A 2B3",2023-01-17 10:00:00-05:00,NaT,0,NaN,NaN,NaN,NaN,NaN,M1A,M1A 2B3,1,Routific Solution Dummy Data - Other 2023-01-1...,2023-01-17,Other,2023-01-17 Other,NaN
1,John Doe,1,Adam Smith,"2 Main St, M1A 2B3",2023-01-17 10:10:00-05:00,2023-01-17 10:20:00-05:00,2,skipped,10:20,Not home,NaN,NaN,M1A,M1A 2B3,0,Routific Solution Dummy Data - Other 2023-01-1...,2023-01-17,Other,2023-01-17 Other,10.0
2,John Doe,2,Beatrice Smith,"2 Main St, M1A 2B4",2023-01-17 10:20:00-05:00,2023-01-17 10:30:00-05:00,2,done,10:30,NaN,NaN,NaN,M1A,M1A 2B4,1,Routific Solution Dummy Data - Other 2023-01-1...,2023-01-17,Other,2023-01-17 Other,10.0
3,John Doe,3,Candice Smith,"2 Main St, M1A 2B5",2023-01-17 10:30:00-05:00,2023-01-17 10:40:00-05:00,2,done,10:40,NaN,NaN,NaN,M1A,M1A 2B5,1,Routific Solution Dummy Data - Other 2023-01-1...,2023-01-17,Other,2023-01-17 Other,10.0
4,John Doe,4,David Smith,"2 Main St, M1A 2B6",2023-01-17 10:40:00-05:00,2023-01-17 10:50:00-05:00,2,done,10:50,NaN,NaN,NaN,M1A,M1A 2B6,1,Routific Solution Dummy Data - Other 2023-01-1...,2023-01-17,Other,2023-01-17 Other,10.0
5,John Doe,5,Ethan Smith,"2 Main St, M1A 2B7",2023-01-17 10:50:00-05:00,2023-01-17 11:00:00-05:00,2,skipped,11:00,Not home,NaN,NaN,M1A,M1A 2B7,0,Routific Solution Dummy Data - Other 2023-01-1...,2023-01-17,Other,2023-01-17 Other,10.0
6,John Doe,999,NaN,"3 Main St, M1A 2B3",2023-01-17 11:30:00-05:00,NaT,10,NaN,NaN,NaN,NaN,NaN,M1A,M1A 2B3,1,Routific Solution Dummy Data - Other 2023-01-1...,2023-01-17,Other,2023-01-17 Other,40.0
7,Mary Doe,0,NaN,"1 Maple St, M1A 2B3",2023-01-17 10:00:00-05:00,NaT,0,NaN,NaN,NaN,NaN,NaN,M1A,M1A 2B3,1,Routific Solution Dummy Data - Other 2023-01-1...,2023-01-17,Other,2023-01-17 Other,NaN
8,Mary Doe,1,Adam Smith,"1 Maple St, M1A 2B4",2023-01-17 10:10:00-05:00,2023-01-17 10:20:00-05:00,2,NaN,NaN,NaN,NaN,NaN,M1A,M1A 2B4,1,Routific Solution Dummy Data - Other 2023-01-1...,2023-01-17,Other,2023-01-17 Other,10.0
9,Mary Doe,2,Beatrice Smith,"1 Maple St, M1A 2B5",2023-01-17 10:20:00-05:00,2023-01-17 10:30:00-05:00,2,NaN,NaN,NaN,NaN,NaN,M1A,M1A 2B5,1,Routific Solution Dummy Data - Other 2023-01-1...,2023-01-17,Other,2023-01-17 Other,10.0


In [43]:
# Write to CSV
clean_path = "../data/clean/"
clean_data.to_csv(clean_path + "Clean_" + file_name)